# 1.- Connectors and libraries

## 1.1.- Libraries

In [ ]:
pip install pymupdf

In [ ]:
pip install tika

In [ ]:
pip install tqdm

In [ ]:
from tqdm import tqdm  # Importa tqdm
#Standard
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#NLP
import fitz
import requests
import nltk
nltk.download('punkt')
import re
from nltk.tokenize import sent_tokenize
import string
from collections import defaultdict
from tika import parser
#Transformers
#Sentiment analysis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from transformers import pipeline  # Hugging Face
# pd.set_option("display.max_colwidth", None)
#version with Alf updated
import re
import requests
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from nltk.tokenize import sent_tokenize
import fitz

## 1.2.- Connectors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install the required libraries
!pip install gspread google-auth

# Import libraries
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default

# Authenticate and create a client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:

# Read/write from Gsheets
from google.colab import auth
from google.auth import default
try:
  import gspread
except ModuleNotFoundError:
  if 'google.colab' in str(get_ipython()):
    %pip install gspread
  import gspread
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
def read_gsheet(spreadsheet_id, sheet_name, first_row_as_column_names=True):
  sht1 = gc.open_by_key(spreadsheet_id)
  worksheet = sht1.worksheet(sheet_name)
  df = pd.DataFrame(worksheet.get_all_values())
  if first_row_as_column_names == True:
    new_header = df.iloc[0]  # Take the first row as the header
    df = df[1:]  # Remove the first row from the dataframe
    df.columns = new_header
  return df
def write_gsheet(spreadsheet_id, sheet_name, df, range=None, clean=True):
  sht1 = gc.open_by_key(spreadsheet_id)
  try:
    worksheet = sht1.worksheet(sheet_name)
  except BaseException:
    worksheet = sht1.add_worksheet(sheet_name,"999","20")
  if clean:
    worksheet.clear()
  if range == None:
    # worksheet.update([df.columns.tolist()]+ df.astype(str).values.tolist())
    worksheet.update([df.columns.tolist()]+ df.fillna('').values.tolist())
  elif range != None:
    # worksheet.update(range, [df.columns.tolist()]+ df.astype(str).values.tolist())
    worksheet.update(range, [df.columns.tolist()]+ df.fillna('').values.tolist())

########################################################################################################################################################################

# spreadsheet_id = ''
# sheet_name = ''
# df_read = read_gsheet(
#     spreadsheet_id = spreadsheet_id,
#     sheet_name = sheet_name,
#     first_row_as_column_names = True)


# spreadsheet_id = ''
# sheet_name = ''
# write_gsheet(
#     spreadsheet_id = spreadsheet_id,
#     sheet_name = sheet_name,
#     df = df,
#     range = f'A1',
#     clean = False)

##########################################################################################################################

# 2- Functions

## 2.1.- PDF reading and handling

### Reading PDF from URL: `read_PDF()`

In [ ]:
def read_pdf(URL):
  res = requests.get(URL)
  doc = fitz.open(stream = res.content, filetype="pdf")
  return doc

### Converting pages in sentences: `to_list()`

The topic analysis (ZSL: Zero-shot-Learning) requires a list as an input

In [ ]:
def to_list(page):
    text_extracted = page.get_text()
    text_normalized = re.sub(r'\s+', ' ', text_extracted)
    sentences = sent_tokenize(text_normalized)
    return sentences

### Converting pages in sentences: `to_dict()`
The sentiment analysis model requires a dictionary as an input

In [ ]:
def to_dict(page):
    text_extracted = page.get_text()
    text_normalized = re.sub(r'\s+', ' ', text_extracted)
    sentences = sent_tokenize(text_normalized)
    dictionary = {
        "sentences": sentences
    }
    return dictionary

## 2.2.- Sentiment analysis

In [ ]:
# Sentiment analysis functions

model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move the model to GPU
model.to('cuda')

def sentiment_analysis(sentence):
    tokenized = tokenizer(sentence, return_tensors="pt", max_length=512, truncation=True)
    # Move the input tensors to GPU
    tokenized = {k: v.to('cuda') for k, v in tokenized.items()}
    # Perform inference on GPU
    prediction = model(**tokenized).logits.argmax().item()
    return model.config.id2label[prediction]

def aggregated_score(analysed):
    list_labels = analysed["labels"]
    total = len(list_labels)
    positive = list_labels.count('POSITIVE')
    negative = list_labels.count('NEGATIVE')
    score = round((positive / total if total > 0 else 0), 2)
    return score

## 2.3.- Topic analysis

In [ ]:
# Topic classification functions

class ZeroShotClassifier:

    def create_zsl_model(self, model_name):
        """ Create the zero-shot learning model. """
        self.model = pipeline("zero-shot-classification", model=model_name, device=0)

    def classify_text(self, text, categories):
        """
        Classify text(s) to the pre-defined categories using a
        zero-shot classification model and return the raw results.
        """
        # Classify text using the zero-shot transformers model
        hypothesis_template = "This text is about {}."
        result = self.model(text, categories, multi_label=True,
                            hypothesis_template=hypothesis_template)
        return result

    def text_labels(self, text, category_dict, cutoff=None):
        """
        Classify a text into the pre-defined categories. If cutoff
        is defined, return only those entries where the score > cutoff
        """
        # Run the model on our categories
        categories = list(category_dict.keys())
        result = self.classify_text(text, categories)

        # Format as a pandas dataframe and add ESG label
        df = pd.DataFrame(result).explode(["labels", "scores"])
        df["ESG"] = df.labels.map(category_dict)

        # If a cutoff is provided, filter the dataframe
        if cutoff:
            df = df[df.scores.gt(cutoff)].copy()
        return df.reset_index(drop=True)

zs_categories = {
  "sustainability": "S",
  "finance": "F"
}

# Define and Create the zero-shot learning model
model_zs_name = "valhalla/distilbart-mnli-12-3"
# a smaller version: "microsoft/deberta-base-mnli"
ZSC = ZeroShotClassifier()
ZSC.create_zsl_model(model_zs_name)

## 2.4.- Scoring function: `from_doc_to_combined_scores`

In [ ]:
def from_doc_to_combined_scores(URL):
    # Extraemos la información en páginas
    doc = read_pdf(URL)
    data_score = []

    for page in tqdm(doc, desc="Processing pages"):  # Añade tqdm aquí para mostrar la barra de progreso
        # Convertimos cada página en oraciones en un diccionario (limpio)
        sentences = to_list(page)

        if not sentences:
            # Si no hay oraciones en la página, agrega un 0 para cada puntaje
            data_score.append({
                'Page': page.number,
                's_score': 0,
                'f_score': 0,
                'sentiment_score': 0
            })
            continue

        # Analizamos el sentimiento para cada oración en la página
        classified = ZSC.text_labels(sentences, zs_categories)

        # Convertimos a float las puntuaciones
        classified['scores'] = classified['scores'].astype('float')

        # Filtramos los puntajes de sostenibilidad
        subset_s = classified[classified.ESG == "S"]
        page_score_s = subset_s.scores.mean() if not subset_s.empty else 0

        # Filtramos los puntajes de finanzas
        subset_f = classified[classified.ESG == "F"]
        page_score_f = subset_f.scores.mean() if not subset_f.empty else 0

        # Analizamos el sentimiento para cada oración en la página
        dict_sentences = to_dict(page)
        dict_sentences["labels"] = [sentiment_analysis(sentence) for sentence in dict_sentences["sentences"]]

        # Calculamos el puntaje agregado
        aggregated_page_score = aggregated_score(dict_sentences)

        data_score.append({
            'Page': page.number,
            's_score': page_score_s,
            'f_score': page_score_f,
            'sentiment_score': aggregated_page_score
        })

    scores = pd.DataFrame(data_score)
    return scores

# 3.- Scoring

In [ ]:
threshold = 10

In [ ]:
input_table = read_gsheet(
    spreadsheet_id = '13vP98GvbJZeRjm-23za6BGD74KdJfpBp4PDWzAIhI8Q',
    sheet_name = 'input',
    first_row_as_column_names = True)
for index, row in input_table.iterrows():
    #Read the output to check if it is already there
    output_completed = read_gsheet(
    spreadsheet_id = '13vP98GvbJZeRjm-23za6BGD74KdJfpBp4PDWzAIhI8Q',
    sheet_name = 'OUTPUT_completed',
    first_row_as_column_names = True)
    #We extract the data needed from every row
    company_id = row['company_id']
    company_name = row['company_name']
    fiscal_year = row['fiscal_year']
    report_link = row['report_link']
    oy = company_name + fiscal_year
    #We check if already there
    print('Checking', company_name, 'report for', fiscal_year)
    if oy in output_completed.OY.unique():
      print('   | Skipped because this company-year is already scored')
    else:
      print('   |Not scored, scoring...')
      print('   |',report_link)
      raw = from_doc_to_combined_scores(URL=report_link)
      df = raw
      df['sustainability_score'] = df['s_score'] / (df['s_score'] + df['f_score'])
      #To check this because it is nan
      norm_sustainability = (df['sustainability_score'] - df['sustainability_score'].min()) / (df['sustainability_score'].max() - df['sustainability_score'].min())
      threshold = 100 - threshold
      sustainability_threshold = np.percentile(norm_sustainability, threshold)
      high_sustainability_df = df[norm_sustainability >= sustainability_threshold]
      sentiment_in_sustainability_sections = round(high_sustainability_df.sentiment_score.mean(),3)
      raw = df
      raw['company_id'] = company_id
      raw['company_name'] = company_name
      raw['fiscal_year'] = fiscal_year
      raw['OY'] = company_name + fiscal_year
      #We append raw to the output_completed
      output_completed = pd.concat([output_completed,raw],ignore_index=True)
      #We write the results updated in Google sheets
      write_gsheet(
      spreadsheet_id = '13vP98GvbJZeRjm-23za6BGD74KdJfpBp4PDWzAIhI8Q',
      sheet_name = 'OUTPUT_completed',
      df = output_completed,
      range = f'A1',
      clean = True)

# 4.- EDA

In [ ]:
import pandas as pd

In [ ]:
scores = read_gsheet(
    spreadsheet_id = '1VHMLOVUY9heQX1qq9eVdIrIh3ewwMQYypye9EvL_QK4',
    sheet_name = 'OUTPUT_Mayo',
    first_row_as_column_names = True)

In [ ]:
df = scores
df = df.fillna(0)

We calculate metrics at OY (Organization - Year) level:

In [ ]:
import pandas as pd

# Supongamos que tu DataFrame se llama df
# Aquí te muestro cómo sería el DataFrame de ejemplo:
# df = pd.DataFrame({
#     'OY': [...],
#     'fiscal_year': [...],
#     'Page': [...],
#     'sentiment_score': [...],
#     'sustainability_score': [...]
# })

def calculate_metrics(df):
    # Seleccionar solo las columnas necesarias
    df = df[['OY', 'fiscal_year', 'Page', 'sentiment_score', 'sustainability_score']]

    # Convertir sentiment_score y sustainability_score a float, manejando valores no convertibles
    df['sentiment_score'] = pd.to_numeric(df['sentiment_score'], errors='coerce')
    df['sustainability_score'] = pd.to_numeric(df['sustainability_score'], errors='coerce')

    # Eliminar filas con NaN en sentiment_score o sustainability_score
    df = df.dropna(subset=['sentiment_score', 'sustainability_score'])

    # Group by OY and calculate mean of sentiment_score and sustainability_score
    mean_scores = df.groupby('OY').agg({
        'sentiment_score': 'mean',
        'sustainability_score': 'mean'
    }).rename(columns={
        'sentiment_score': 'mean_sentiment_score',
        'sustainability_score': 'mean_sustainability_score'
    })

    # Calculate mean sentiment_score for the top 10% pages by sustainability_score for each OY
    top_10_sentiment = df.groupby('OY').apply(
        lambda x: x.nlargest(int(len(x) * 0.1), 'sustainability_score')['sentiment_score'].mean()
    ).reset_index().rename(columns={0: 'top_10_percent_sentiment_score'})

    # Merge the results into a single DataFrame
    result = mean_scores.merge(top_10_sentiment, on='OY')

    return result

# Ejemplo de uso
# df = pd.read_csv('path_to_your_file.csv')
result_df = calculate_metrics(df)

We add Taxonomy data:

In [ ]:
taxonomy = read_gsheet(
    spreadsheet_id = '1VHMLOVUY9heQX1qq9eVdIrIh3ewwMQYypye9EvL_QK4',
    sheet_name = 'INPUT',
    first_row_as_column_names = True)

In [ ]:
taxonomy = taxonomy[['industry','company_name','fiscal_year', 'report_type','revenue_alignment','CAPEX_alignment']]

In [ ]:
names_id = df[['company_name','fiscal_year','OY']]
names_id = names_id.drop_duplicates()

In [ ]:
result_df = result_df.merge(names_id, on=['OY'])

In [ ]:
result_df = result_df.merge(taxonomy,on=['company_name','fiscal_year'])

We plot:

We check correlation between variables: sentiment score and revenue alignment and capex alignment

In [ ]:
check_revenue = result_df[['OY', 'mean_sentiment_score','revenue_alignment']]
check_revenue['mean_sentiment_score'] = pd.to_numeric(check_revenue['mean_sentiment_score'], errors='coerce')
check_revenue['revenue_alignment'] = pd.to_numeric(check_revenue['revenue_alignment'], errors='coerce')
check_revenue['revenue_alignment'] = check_revenue['revenue_alignment']/100

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Establecer el estilo de seaborn
sns.set(style="ticks")

# Crear el gráfico de dispersión
plt.figure(figsize=(8,8))
scatter_plot = sns.scatterplot(data=check_revenue, x='revenue_alignment', y='mean_sentiment_score', palette='viridis',
                               sizes=(300,300),s=100, alpha=0.6, edgecolor=None)

plt.xlabel('Sustainability Performance', fontsize=14)
plt.ylabel('Sentiment Analysis', fontsize=14)

plt.xlim(0, 1)  # Cambia los valores según tus datos
plt.ylim(0, 1)   # Cambia los valores según tus datos

# Mostrar el gráfico
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurar que las columnas necesarias son de tipo float
result_df['mean_sentiment_score'] = pd.to_numeric(result_df['mean_sentiment_score'], errors='coerce')
result_df['revenue_alignment'] = pd.to_numeric(result_df['revenue_alignment'], errors='coerce')

# # Dividir revenue_alignment entre 100
# result_df['revenue_alignment'] = result_df['revenue_alignment'] / 100

# Reestructurar el DataFrame para tener las métricas en la misma columna
melted_df = result_df.melt(id_vars='company_name', value_vars=['mean_sentiment_score', 'revenue_alignment'],
                           var_name='metric', value_name='value')

# Crear el box plot combinado
plt.figure(figsize=(14, 4))
sns.boxplot(x='company_name', y='value', hue='metric', data=melted_df)
plt.title('Mean Sentiment Score and Revenue Alignment by Company')
plt.xlabel('Company')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.legend(title='Metric')

# Añadir líneas discontinuas entre las compañías
num_companies = melted_df['company_name'].nunique()
for i in range(num_companies - 1):
    plt.axvline(x=i + 0.5, color='gray', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

# 5.- EDA para las secciones de sostenibilidad

In [ ]:
check_revenue_sust = result_df[['OY', 'revenue_alignment','top_10_percent_sentiment_score']]
check_revenue_sust['mean_sentiment_score'] = pd.to_numeric(check_revenue_sust['top_10_percent_sentiment_score'], errors='coerce')
check_revenue_sust['revenue_alignment'] = pd.to_numeric(check_revenue_sust['revenue_alignment'], errors='coerce')
check_revenue_sust['revenue_alignment'] = check_revenue_sust['revenue_alignment']/100

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Establecer el estilo de seaborn
sns.set(style="ticks")

# Crear el gráfico de dispersión
plt.figure(figsize=(8,8))
scatter_plot = sns.scatterplot(data=check_revenue_sust, x='revenue_alignment', y='top_10_percent_sentiment_score', palette='viridis',
                               sizes=(300,300),s=100, alpha=0.6, edgecolor=None)

plt.xlabel('Sustainability Performance', fontsize=14)
plt.ylabel('Sentiment Analysis', fontsize=14)

plt.xlim(0, 1)  # Cambia los valores según tus datos
plt.ylim(0, 1)   # Cambia los valores según tus datos

# Mostrar el gráfico
plt.show()

In [ ]:
from scipy.stats import pearsonr

# Correlación de Pearson para revenue_alignment y mean_sentiment_score
corr_revenue, p_value_revenue = pearsonr(check_revenue_sust['revenue_alignment'], check_revenue_sust['top_10_percent_sentiment_score'])

print(f'Pearson correlation (Revenue): {corr_revenue}, p-value: {p_value_revenue}')


In [ ]:
import statsmodels.api as sm
# Análisis de regresión para revenue_alignment y mean_sentiment_score
X_revenue = sm.add_constant(check_revenue_sust['revenue_alignment'])
model_revenue = sm.OLS(check_revenue_sust['top_10_percent_sentiment_score'], X_revenue).fit()

print(model_revenue.summary())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import uniform_filter1d
import matplotlib.cm as cm

# Supongamos que tu DataFrame se llama df
# Ejemplo de DataFrame:
# df = pd.DataFrame({
#     'company_id': [...],
#     'company_name': [...],
#     'fiscal_year': [...],
#     'OY': [...],
#     'Page': [...],
#     's_score': [...],
#     'f_score': [...],
#     'sentiment_score': [...],
#     'sustainability_score': [...]
# })

# Convertir 'Page' y 'sentiment_score' a numérico
examples['Page'] = pd.to_numeric(examples['Page'], errors='coerce')
examples['sustainability_score'] = pd.to_numeric(examples['sustainability_score'], errors='coerce')

# Eliminar filas con valores nulos en 'Page' o 'sentiment_score'
examples = examples.dropna(subset=['Page', 'sustainability_score'])

examples['fiscal_year'] = pd.Categorical(examples['fiscal_year'], categories=sorted(examples['fiscal_year'].unique()), ordered=True)

# Función para aplicar el suavizado
def smooth(data, window_size=15):
    return uniform_filter1d(data, size=window_size)

# Extraer colores de la paleta 'viridis'
viridis = cm.get_cmap('viridis')
original_color = viridis(0.7)  # Color para la línea original
smooth_color = viridis(0.2)    # Color para la línea suavizada
mean_color = viridis(0)      # Color para la línea de promedio

# Ajustar el estilo y tamaño de fuente
sns.set_context("talk", font_scale=1.2)

# Crear la matriz de gráficos sin normalizar las páginas
g = sns.FacetGrid(examples, row='company_name', col='fiscal_year', margin_titles=True, height=7, aspect=1, palette='viridis')

# Mapear tanto la línea original como la suavizada
g.map(sns.lineplot, 'Page', 'sustainability_score', color=original_color, alpha=0.5)
g.map(lambda x, y, **kwargs: plt.plot(x, smooth(y), color=smooth_color, linewidth=2), 'Page', 'sustainability_score')

# Función para agregar la línea de promedio
def add_mean_line(data, **kwargs):
    plt.axhline(y=data['sustainability_score'].mean(), color=mean_color, linestyle='--', linewidth=1)

# Añadir la línea de promedio a cada gráfico
g.map_dataframe(add_mean_line)

# Añadir títulos y ajustar el layout
g.set_axis_labels('Page', 'Sustainability topic Score')
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.fig.subplots_adjust(top=0.95)


plt.show()